In [36]:
import pandas as pd
import numpy as np
from measurement_import import PingImport, TracerouteImport

import warnings
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
ping_import = PingImport()
df_ping = ping_import.read_dataset('measurement_data/ping-measurement_1.json')
df_ping

,probe_id,created,rtt_min,rtt_average,packets_sent,packets_received
0,6025,2021-06-07 00:02:11+00:00,149.195256,149.218,3,3
1,6030,2021-06-07 00:02:21+00:00,166.409528,166.576,3,3
2,6031,2021-06-07 00:02:22+00:00,149.048305,149.319,3,3
3,6039,2021-06-07 00:02:21+00:00,170.077631,170.098,3,3
4,6041,2021-06-07 00:02:17+00:00,173.588153,173.648,3,3
...,...,...,...,...,...,...
678089,7006,2021-06-09 23:58:21+00:00,147.811359,147.874,3,3
678090,7007,2021-06-09 23:58:17+00:00,208.128699,208.156,3,3
678091,7008,2021-06-09 23:58:09+00:00,162.276314,162.362,3,3
678092,7009,2021-06-09 23:58:08+00:00,219.913387,219.927,3,3


In [38]:
traceroute_import = TracerouteImport()
df_traceroute = traceroute_import.read_dataset('measurement_data/single-probe-anchor-traceroute_1.json')
df_traceroute

,probe_id,created,total_hops,pre_entry_hop_min_rtt,pre_entry_hop_ip,pre_entry_as
0,684,2022-01-11 11:37:15+00:00,15,306.375,114.5.178.138,4761
